<a href="https://colab.research.google.com/github/HSSunag/Final-Project/blob/master/augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
 
# Accessing My Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
print(len(os.listdir('drive/My Drive/Datasets/Category1')))
print(len(os.listdir('drive/My Drive/Datasets/Category2')))
print(len(os.listdir('drive/My Drive/Datasets/Category3')))

270
225
234


In [0]:
try:
    #YOUR CODE GOES HERE
    os.mkdir('drive/My Drive/Data')
    training = os.path.join('drive/My Drive/Data','training')
    testing = os.path.join('drive/My Drive/Data','testing')
    os.mkdir(training)
    os.mkdir(testing)
    cat1 = os.path.join(training,'cat1')
    cat2 = os.path.join(training,'cat2')
    cat3 = os.path.join(training,'cat3')
    cat12 = os.path.join(testing,'cat1')
    cat22 = os.path.join(testing,'cat2')
    cat32 = os.path.join(testing,'cat3')
    os.mkdir(cat1)
    os.mkdir(cat2)
    os.mkdir(cat3)
    os.mkdir(cat12)
    os.mkdir(cat22)
    os.mkdir(cat32)
except OSError:
    pass

In [0]:
import shutil
from shutil import copyfile

In [0]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2020-05-16 03:02:40--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c16::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   197MB/s    in 0.4s    

2020-05-16 03:02:40 (197 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [0]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "drive/My Drive/Data/training"
training_datagen = ImageDataGenerator(
      rescale = 1./255)

VALIDATION_DIR = "drive/My Drive/Data/testing"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

Found 584 images belonging to 3 classes.
Found 145 images belonging to 3 classes.


In [0]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (3, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 



model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=3, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

Epoch 1/3
20/20 [==============================] - 696s 35s/step - loss: 8.1177 - accuracy: 0.7684 - val_loss: 2.9978 - val_accuracy: 0.4834
Epoch 2/3
20/20 [==============================] - 686s 34s/step - loss: 0.1644 - accuracy: 0.9696 - val_loss: 8.7798 - val_accuracy: 0.6347
Epoch 3/3
20/20 [==============================] - 687s 34s/step - loss: 1.2121 - accuracy: 0.9054 - val_loss: 3.8952 - val_accuracy: 0.4760


In [0]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = os.listdir(SOURCE)
    #random.sample(files, len(files))
    training_files = SPLIT_SIZE*(len(files))
    for i in range(0,len(files)):
        if os.path.getsize(os.path.join(SOURCE,files[i])) == 0:
            continue
        if i < training_files:
            copyfile(os.path.join(SOURCE,files[i]),os.path.join(TRAINING,files[i]))
        if i>= training_files:
            copyfile(os.path.join(SOURCE,files[i]),os.path.join(TESTING,files[i]))
    
# YOUR CODE STARTS HERE
# YOUR CODE ENDS HERE


CAT1_SOURCE_DIR = "drive/My Drive/Datasets/Category1/"
CAT2_SOURCE_DIR = "drive/My Drive/Datasets/Category2/"
CAT3_SOURCE_DIR = "drive/My Drive/Datasets/Category3/"
TRAINING_CAT1_DIR = "drive/My Drive/Data/training/cat1/"
TRAINING_CAT2_DIR = "drive/My Drive/Data/training/cat2/"
TRAINING_CAT3_DIR = "drive/My Drive/Data/training/cat3/"
TESTING_CAT1_DIR = "drive/My Drive/Data/testing/cat1/"
TESTING_CAT2_DIR = "drive/My Drive/Data/testing/cat2/"
TESTING_CAT3_DIR = "drive/My Drive/Data/testing/cat3/"


split_size = .8
split_data("drive/My Drive/Datasets/Category1/", "drive/My Drive/Data/training/cat1/", "drive/My Drive/Data/testing/cat1/", split_size)

In [0]:
split_data("drive/My Drive/Datasets/Category2/", "drive/My Drive/Data/training/cat2/", "drive/My Drive/Data/testing/cat2/", split_size)

In [0]:
split_data("drive/My Drive/Datasets/Category3/", "drive/My Drive/Data/training/cat3/", "drive/My Drive/Data/testing/cat3/", split_size)

In [0]:
print(len(os.listdir('drive/My Drive/Data/training/cat1/')))
print(len(os.listdir('drive/My Drive/Data/training/cat2/')))
print(len(os.listdir('drive/My Drive/Data/training/cat3/')))
print(len(os.listdir('drive/My Drive/Data/testing/cat1/')))
print(len(os.listdir('drive/My Drive/Data/testing/cat2/')))
print(len(os.listdir('drive/My Drive/Data/testing/cat3/')))

216
180
188
54
46
45


In [0]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "drive/My Drive/Data/training"
training_datagen = ImageDataGenerator(
      rescale = 1./255)

VALIDATION_DIR = "drive/My Drive/Data/testing"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)



Found 584 images belonging to 3 classes.
Found 145 images belonging to 3 classes.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
______________________________________

In [0]:
os.chdir('drive/My Drive')

In [0]:
model.save('areca1.h5')